This solution is hideous - but I never want to touch this puzzle again

In [1]:
import sys
sys.path.append("..")

In [2]:
from enum import Enum

import re

from resources.utils import get_puzzle_input

In [3]:
class Direction(Enum):
    UP = 0
    LEFT = 1
    RIGHT = 2
    DOWN = 3
    
ordered_directions = (
    Direction.UP,
    Direction.LEFT,
    Direction.RIGHT,
    Direction.DOWN,
)
    
MOVE_MAP = {
    Direction.LEFT: (-1, 0),
    Direction.RIGHT: (1, 0),
    Direction.UP: (0, -1),
    Direction.DOWN: (0, 1),
}

def direction_move(pos, direction):
    offset_x, offset_y = MOVE_MAP[direction]
    return pos[0] + offset_x, pos[1] + offset_y

### Part 1

In [4]:
test_input = """x=495, y=2..7
y=7, x=495..501
x=501, y=3..7
x=498, y=2..4
x=506, y=1..2
x=498, y=10..13
x=504, y=10..13
y=13, x=498..504""".split('\n')

In [5]:
def parse_input(lines):
    line_re = re.compile('^([xy])=([0-9]+), [xy]=([0-9]+)..([0-9]+)')
    clay = []

    for line in lines:
        match = line_re.match(line)
        if not match:
            raise ValueError('Cannot parse line: {}'.format(line))
        
        first_coord, single_val, range_start, range_end = match.groups()
        
        clay_range = range(int(range_start), int(range_end) + 1)
        single_val = int(single_val)
        
        if first_coord=='x':
            clay.extend([(single_val, pos) for pos in clay_range])
        else:
            clay.extend([(pos, single_val) for pos in clay_range])
            
    return set(clay)   


```
   44444455555555
   99999900000000
   45678901234567
 0 ......+.......
 1 ............#.
 2 .#..#.......#.
 3 .#..#..#......
 4 .#..#..#......
 5 .#.....#......
 6 .#.....#......
 7 .#######......
 8 ..............
 9 ..............
10 ....#.....#...
11 ....#.....#...
12 ....#.....#...
13 ....#######...
```

In [6]:
test_clay = parse_input(test_input)

In [7]:
def is_sand(pos, max_y, clay, water):
    return (
        pos[1] <= max_y and
        pos not in clay and
        pos not in water
    )

In [8]:
def fill_layer(pos, max_y, clay, water):
    drainage_points = []
    layer_points = {pos}
    
    down = direction_move(pos, Direction.DOWN)
    
    if pos[1] >= max_y:
        return layer_points, []

    if is_sand(down, max_y, clay, water):
        return layer_points, [down]

    for direction in Direction.LEFT, Direction.RIGHT:
        curr_pos = pos
        while True:
            curr_pos = direction_move(curr_pos, direction)
            if not is_sand(curr_pos, max_y, clay, water):
                break
            layer_points.add(curr_pos)
            down = direction_move(curr_pos, Direction.DOWN)
            if is_sand(down, max_y, clay, water):
                drainage_points.append(down)
                break
    
    if not drainage_points:
        water |= layer_points
    
    return layer_points, drainage_points

In [9]:
def drip_position(start, max_y, clay, water):
    pos = start
    path = set()
    drip_stack = []
    drip_fill_points = []
    final_positions = set()
    first_decision_point = None

    while True:
        visited, drainage_points = fill_layer(pos, max_y, clay, water)
        path |= visited

        if not drainage_points:
            break

        if len(drainage_points) == 2:
            drip_stack.append(drainage_points[1])

        pos = drainage_points[0]
        path.add(pos)

    return drip_stack, path

In [10]:
def pour(source, clay):
    max_y = max(y for _, y in clay)
    water = set()
    reached = set()
    drip_num = 0
    drip_stack = [source]
    prev_final_position = None
    fork_points = set()

    while(drip_stack):
        start = drip_stack[-1]

        if drip_num % 100 == 0:
            print('Pouring drip {}, reached: {}'.format(
                drip_num,
                len(reached)
            ))

        num_previous_reached = len(reached)
        num_previous_water = len(water)

        forks, path = drip_position(
            start=start,
            max_y=max_y,
            clay=clay,
            water=water
        )
        new_forks = [f for f in forks if f not in fork_points]

        fork_points |= set(new_forks)
        drip_stack.extend(new_forks)
        reached |= path

        if (
            len(reached) == num_previous_reached and
            len(water) == num_previous_water and
            not new_forks
        ):
            #print('popped')
            drip_stack.pop()

        drip_num += 1
    
    return reached, water  

In [11]:
def plot_grid(reached, sand, water, file=None):
    min_x = None
    min_y = None
    max_x = None
    max_y = None
    
    for x, y in sand:
        if max_x is None or x > max_x:
            max_x = x
        if min_x is None or x < min_x:
            min_x = x
        if max_y is None or y > max_y:
            max_y = y
        if min_y is None or y < min_y:
            min_y = y
    
    with open(file, 'w') as fh:
        for y in range (min_y - 2, max_y + 2):
            line = []
            for x in range (min_x - 3, max_x + 3):
                point = x, y
                if point in water:
                    char = '~'
                elif point in sand:
                    char = '#'
                elif point in reached:
                    char = '|'
                else:
                    char = '.'

                line.append(char)
            fh.write(''.join(line))
            fh.write('\n')

In [12]:
test_reached, test_water = pour((500, 0), test_clay)
len(test_reached)

Pouring drip 0, reached: 0


58

In [13]:
plot_grid(test_reached, test_clay, test_water, '/tmp/test.txt')

In [14]:
puzzle_input = get_puzzle_input('/tmp/day_17.txt')

In [15]:
puzzle_clay = parse_input(puzzle_input)

In [16]:
reached, water = pour((500, 0), puzzle_clay)

Pouring drip 0, reached: 0
Pouring drip 100, reached: 2006
Pouring drip 200, reached: 3731
Pouring drip 300, reached: 7344
Pouring drip 400, reached: 9007
Pouring drip 500, reached: 10871
Pouring drip 600, reached: 12678
Pouring drip 700, reached: 14438
Pouring drip 800, reached: 15670
Pouring drip 900, reached: 18151
Pouring drip 1000, reached: 21937
Pouring drip 1100, reached: 27508
Pouring drip 1200, reached: 29137
Pouring drip 1300, reached: 30463


In [17]:
len(reached)

31955

In [18]:
#Aggghhh!!!! Missed this line in the text
top = min(y for _, y in puzzle_clay)
top

6

In [19]:
len([_ for (_, y) in list(reached) if y >= top])

31949

In [20]:
plot_grid(reached, puzzle_clay, water, '/tmp/puzzle_solution.txt')

### Part 2

In [21]:
len(water)

26384

Failing on this shape for some time due to breaking out early if we've
already hit the paths. Needed to check whether we had identified further
still water

```
0
10123456789012345678901234
2....#......|.............
3....#......|.........#...
4....#......|.........#...
5....#||||||||||......#...
6....#~~~~~~~~#~#.....#...
7....#~~~~~~~~#~#.....#...
8....#~~~~~~~~#~#.....#...
9....#~~~~~~~~###.....#...
0....#~~~~~~~~~~~~~~~~#...
1....#~~~~~~~~~~~~~~~~#...
2....#~~~~~~~~~~~~~~~~#...
3....#~~~~~~~~~~~~~~~~#...
4....##################...
5.........................
```

In [22]:
problem_shape_input=[
    'x=4, y=2..14',
    'y=14, x=4..21',
    'x=21, y=3..14',
    'x=13, y=6..9',
    'y=9, x=13..15',
    'x=15, y=6..9'
]
problem_shape = parse_input(problem_shape_input)
problem_shape_reached, problem_shape_water = pour((11, 0), problem_shape)
len(problem_shape_reached), len(problem_shape_water)
plot_grid(
    problem_shape_reached,
    problem_shape,
    problem_shape_water,
    '/tmp/problem_shape.txt'
)

Pouring drip 0, reached: 0
